# Nestle - Zadanie Rekturacyjne
## Business Case - Data Science Hub
### Autor: Szymon Baczyński

## 1. Wczytanie danych i przegląd
### Pliki treningowe: X_train.csv & Y_train.csv
### Pliki testowe: X_test.csv & Y_test.csv

In [ ]:
import pandas as pd
import random
import numpy as np
import xgboost as xgb
from datetime import datetime
import requests
from matplotlib import pyplot
import seaborn as sns
import IPython.display as Disp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

sns.set_theme()

In [ ]:
# Wczytanie danych i szybki przeglad
X_train = pd.read_csv("X_train.csv", delimiter= ";", decimal=",")
X_train.info()
X_train.head()

In [ ]:
# Rozmiar i nazwy kolumn
X_train.shape
X_train.columns

In [ ]:
# ID produktow w danych treningowych
X_train.key.unique()

# Ilosc unikalnych produktow
X_train.key.nunique()

In [ ]:
# Sprawdzenie zakresu dat
X_train.date.unique()

In [ ]:
# Sprawdzenie co sie dzieje w danych - nowe produkty (w czasie) + zmienna ilosc danych dla poszczegolnych produktow
X_train.groupby('key').count()

In [ ]:
# Ile jest wartosci w poszczegolnych kolumnach -> nie ma problemu w 'key' i 'date'
X_train.isnull().sum()
# i ilosc wartosci nieliczbowych dla poszczegolnych produktow
# pd.concat([X_train.loc[:,'key'], X_train.isnull().sum(axis=1)], axis=1).groupby('key').sum()

In [ ]:
# Sprawdzmy przy okazji dane "target" -> "Y"
Y_train = pd.read_csv("Y_train.csv", delimiter= ";", decimal=",")
Y_train.info()
Y_train.head()

In [ ]:
# Zobaczmy ile jest wartosci null czy NaN
Y_train.isnull().sum()
Y_train[Y_train.isnull().any(axis=1)]
Y_train[Y_train.isnull().any(axis=1)].groupby('key').count()

In [ ]:
# Dane "Y" - produkty tez maja rozna ilosc dat
Y_train.groupby('key').count()

In [ ]:
# Sprawdzmy czy ilosc dat dla X i Y w zbiorze treningowym sie zgadzaja
all(X_train.groupby('key').count().date == Y_train.groupby('key').count().date)

In [ ]:
# Zduplikowane wiersze "key" i "date"?
X_train.duplicated(subset=['key','date']).any()
Y_train.duplicated(subset=['key','date']).any()

## 2. Data Wrangling - zróbmy z tego dane, które da się czytać

In [ ]:
url = 'https://miro.medium.com/max/400/0*TXDoF8j-D3LuGbHP.jpg'
Disp.Image(requests.get(url).content)

In [ ]:
# Naprawmy kolumne z datami
X_train['date'] = pd.to_datetime(X_train['date'], format='%d%b%Y')
X_train.info()
X_train.head()

In [ ]:
X_train.iloc[:,2:].describe()

In [ ]:
# Plot some variables 'x' for random keys to show missing data
rand_keys = random.choices(X_train['key'].unique(), k=5)
rand_keys

for keys in rand_keys:
    pyplot.plot(X_train.loc[X_train['key'] == keys].date, X_train.loc[X_train['key'] == keys].iloc[:,3:10]);
    pyplot.legend(X_train.iloc[:,3:10].columns);
    pyplot.show();

Notatki:
- Widać ubytki w danych, często dość kluczowe - można by zastosować średnią kroczącą, w celu uzupełnienia 
- Niestety takie uzupełnianie może nie być prawdziwe - warto zapytać klienta czy nie ma tych danych


In [ ]:
X_train_copy = X_train.copy(deep=True)

In [ ]:
# Czyszczenie wartosci NaN / NULL

# X_train_copy.fillna(0.0)
# X_train_copy = X_train_copy.fillna(X_train_copy.rolling(20,min_periods=1).mean())

col_names = X_train.iloc[:,2:].columns
X_train_copy[col_names] = X_train_copy.groupby('key')[col_names]\
    .transform(lambda x: x.fillna(x.rolling(3,min_periods=1).mean()))

X_train_copy.fillna(0.0, inplace=True)
# Znalezc jak sie robi fillna z 0 na poczatku a pozniej innymi wartosciami


In [ ]:
X_train.loc[X_train['key'] == 683].isnull().sum().sum()
X_train_copy.loc[X_train_copy['key'] == 683].isnull().sum().sum()

In [ ]:
# Uzupelnienie wartosci "Y"

In [ ]:
# Korelacja
# https://medium.com/@sebastiannorena/finding-correlation-between-many-variables-multidimensional-dataset-with-python-5deb3f39ffb3
# https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
# https://stackoverflow.com/questions/42128462/in-python-how-to-do-correlation-between-multiple-columns-more-than-2-variables
# https://realpython.com/numpy-scipy-pandas-correlation-python/
